In [1]:
model_path = "craft_mlt_25k.pth"

In [2]:
import torch
import onnx
from craft import CRAFT # Class from Craft Repository (https://github.com/clovaai/CRAFT-pytorch) 
from collections import OrderedDict

In [3]:
def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

In [4]:
# Load the model and its weights
model = CRAFT()
model.load_state_dict(copyStateDict(torch.load(model_path, map_location='cpu')))

# Set the model to evaluation mode
model.eval()

In [5]:
dummy_input = torch.randn(1, 3, 768, 768, device="cpu")

In [6]:
input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
output_names = [ "output1" ]

In [7]:
torch.onnx.export(model, dummy_input, "craft_mlt_25k.onnx", verbose=True, input_names=input_names, output_names=output_names)